In [27]:
classes_name =  ["aeroplane", "bicycle", "bird", "boat", "bottle", 
                 "bus", "car", "cat", "chair", "cow", "diningtable", 
                 "dog", "horse", "motorbike", "person", "pottedplant", 
                 "sheep", "sofa", "train","tvmonitor"]

training_data_file = open("./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/pascal_voc_training_data.txt", "r")
for i, line in enumerate(training_data_file):
    if i >5:
        break
    line = line.strip()
    print(line)

000005.jpg 263 211 324 339 8 165 264 253 372 8 5 244 67 374 8 241 194 295 299 8 277 186 312 220 8
000007.jpg 141 50 500 330 6
000009.jpg 69 172 270 330 12 150 141 229 284 14 285 201 327 331 14 258 198 297 329 14
000012.jpg 156 97 351 270 6
000016.jpg 92 72 305 473 1
000017.jpg 185 62 279 199 14 90 78 403 336 12


In [57]:
import imgaug.augmenters as iaa
from imgaug import BoundingBoxesOnImage, BoundingBox
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import shutil

def augmentation(num_of_aug, chosen_class, clear):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # Flip horizontally
        iaa.Flipud(0.5),
        iaa.Multiply((0.8, 1.2)),  # Random brightness
        iaa.Crop(percent=(0, 0.1)),  # Random cropping
        iaa.GaussianBlur(sigma=(0.0, 1.5)),
        iaa.Dropout(p=0.05),  # Randomly drop (black out) 5% of the pixels
        iaa.Grayscale(alpha=(0.0, 1.0))  # Convert to grayscale with random intensity
    ])

    for t in range(num_of_aug):
        bboxes_aug_arr = []

        # parameters
        chosen_class_name = classes_name[chosen_class]
        print(f'The chosen class is {chosen_class_name}')
        dir_name = f'./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/VOCdevkit_train/VOC2007/Augmented Images/{chosen_class_name}'
        training_data_file = open("./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/pascal_voc_training_data.txt", "r")

        # Remove all contents within the directory
        os.makedirs(dir_name, exist_ok=True)
        if(clear):
            for filename in os.listdir(dir_name):
                file_path = os.path.join(dir_name, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)  # Remove file or symbolic link
                except Exception as e:
                    print(f'Failed to delete {file_path}. Reason: {e}')

            print(f"All contents within {dir_name} have been removed.")

        # get file length
        files = [f for f in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, f))]
        file_count = len(files)

        # perfrom augmentation
        if clear: aug_img_idx = 0
        else: aug_img_idx = file_count

        for line in training_data_file:

            #  get info of each image from the .txt file
            line = line.strip()
            ss = line.split(' ')
            image = cv2.imread(f"2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/VOCdevkit_train/VOC2007/JPEGImages/{ss[0]}")
            bboxes_coords = []
            labels = []

            # record the bounding box and the label
            for idx in range(1, 101, 5):
                try:
                    bboxes_coords.append((BoundingBox(float(ss[idx]), float(ss[idx+1]), float(ss[idx+2]), float(ss[idx+3]), float(ss[idx+4]))))
                    labels.append(int(ss[idx+4]))
                except:
                    break

            # check if the chosen class is in the image
            if chosen_class not in labels or 14 in labels:
                continue
            aug_img_idx+=1
            
            bboxes_on_image = BoundingBoxesOnImage(bboxes_coords, shape=image.shape)

            image_aug, bboxes_aug = seq(image=image, bounding_boxes=bboxes_on_image)
            bboxes_aug_arr.append(bboxes_aug)
            cv2.imwrite(f'{dir_name}/{aug_img_idx:06d}.jpg', image_aug)


        # write the txt file of the augmented images
        if clear: aug_img_idx = 0
        else: aug_img_idx = file_count

        os.makedirs(f'./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data', exist_ok=True)

        # Define the file path
        file_path = f'./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/{chosen_class_name}.txt'
        # Check if the file exists
        if (not os.path.exists(file_path)) or clear:
            # Create the file (in append mode)
            with open(file_path, 'w') as file:
                print(f"The file {file_path} was created.")
        else:
            print(f"The file {file_path} already exists.")

        with open(file_path, 'a') as file:
            for img_info in bboxes_aug_arr:

                aug_img_idx+=1
                file.write(f"{aug_img_idx:06d}.jpg ")

                for bbox in img_info:
                    xmin, ymin, xmax, ymax, label = np.round(bbox.x1).astype(int), np.round(bbox.y1).astype(int), np.round(bbox.x2).astype(int), np.round(bbox.y2).astype(int),  np.round(bbox.label).astype(int)
                    file.write(f"{xmin} {ymin} {xmax} {ymax} {label} ")
                    
                file.write('\n')

        
    
    

In [99]:
chosen_class = 18
iter = 10
clear = False

In [102]:
# 看各個obj有幾多少個
obj_list = []
obj_dict = {}

training_data_file = open("./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/pascal_voc_training_data.txt", "r")
for line in training_data_file:
    line = line.strip()
    ss = line.split(' ')
    
    for idx in range(5, 101, 5):
        try:
            obj_num = ss[idx]
        except:
            break
        if obj_num in obj_dict:
            obj_dict[obj_num] += 1
        else:
            obj_dict[obj_num] = 1

for class_name in classes_name:
    try:
        augmented_data = open(f"./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/{class_name}.txt", "r")

    except:
        continue

    for line in augmented_data:
        line = line.strip()
        ss = line.split(' ')
        
        for idx in range(5, 101, 5):
            try:
                obj_num = ss[idx]
            except:
                break
            if obj_num in obj_dict:
                obj_dict[obj_num] += 1
            else:
                obj_dict[obj_num] = 1

print(f'Object counts of {classes_name[chosen_class]} is {obj_dict[str(chosen_class)]}')
print(obj_dict)

Object counts of train is 3240
{'8': 4318, '6': 4373, '12': 3061, '14': 5318, '1': 3127, '7': 3343, '11': 3450, '18': 3240, '0': 347, '10': 623, '19': 747, '2': 613, '4': 933, '13': 414, '15': 1267, '3': 497, '17': 1095, '16': 556, '9': 391, '5': 451}


In [101]:
augmentation(iter, chosen_class, clear)

The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-object-detection/2024-datalab-cup2-object-detection/augmented training data/train.txt already exists.
The chosen class is train
The file ./2024-datalab-cup2-obj